In [14]:
__file__

NameError: name '__file__' is not defined

In [20]:
with open("prompts/pddl_format.txt") as f:
    prompt = f.read().rstrip()+"\n"

In [4]:

prompt_edit = """
    Please provide the output in JSON format, including the edit suggestions for a domain file as 'df' and the edit suggestions for a problem file as 'pf'. 
    The output format should be: {{"df": "...", "pf": "..."}}
    You will modify the following df and pf using add, delate, and replace operations (in a JSON format). 
    You SHOULD NOT provide a domain file and a problem file directly.
    This is the structure for df edit file, remember to add bracket:
    {{
    "predicates": {{
        "add": ["(predicates to add)"],
        "replace": {{"(old)": "(new)"}},
        "delete": ["(predicates to delete)"]
        }},
    "action": {{
        "open-door": {{
            "precondition": ["(entire full new precondition for open-door)"], # directly replace the whole precondition
            "effect": ["(entire full new effect for open-door)"] # so as effect
            }},
        "move": {{
            "precondition": []
            "effect": []
            }}
        }}
    }}
    This is the structure for pf edit file:
    {{
    "objects": {{
        "add": [],
        "replace": {{}},
        "delete": []
        }},
    "init": {{
        "add": [],
        "replace": {{}},
        "delete": []
        }},
    "goal": ["(entire full new goal)"]
    }}
"""
print(prompt_edit.format())



    Please provide the output in JSON format, including the edit suggestions for a domain file as 'df' and the edit suggestions for a problem file as 'pf'. 
    The output format should be: {"df": "...", "pf": "..."}
    You will modify the following df and pf using add, delate, and replace operations (in a JSON format). 
    You SHOULD NOT provide a domain file and a problem file directly.
    This is the structure for df edit file, remember to add bracket:
    {
    "predicates": {
        "add": ["(predicates to add)"],
        "replace": {"(old)": "(new)"},
        "delete": ["(predicates to delete)"]
        },
    "action": {
        "open-door": {
            "precondition": ["(entire full new precondition for open-door)"], # directly replace the whole precondition
            "effect": ["(entire full new effect for open-door)"] # so as effect
            },
        "move": {
            "precondition": []
            "effect": []
            }
        }
    }
    This is the s

hell
{var2}

hell
{{var2}}



In [32]:
prompt_format = """
    Please provide the output in strict JSON format, without any additional text or explanation, including a PDDL domain file as 'df' and a PDDL problem file as 'pf'. 
    The format should strictly be:
        {{
        "df": "...",
        "pf": "..."
        }}

    {prompt}
"""
print(prompt_format.format(prompt = "This is a variable"))


    Please provide the output in strict JSON format, without any additional text or explanation, including a PDDL domain file as 'df' and a PDDL problem file as 'pf'. 
    The format should strictly be:
        {
        "df": "...",
        "pf": "..."
        }

    This is a variable



In [29]:
print(prompt)

Please provide the output in strict JSON format, without any additional text or explanation, including a PDDL domain file as 'df' and a PDDL problem file as 'pf'. 
The format should strictly be:
    {
    "df": "...",
    "pf": "..."
    }


{var}



In [30]:
prompt.format(var = "This is a variable")

KeyError: '\n    "df"'

In [25]:
import torch
import gc

In [27]:
gc.collect()

20

In [ ]:


def clear_cuda_memory(model_name):
    """Clears a specific model from cache and frees GPU memory."""
    global _kani_cache, _hf_engine_cache

    if model_name in _kani_cache:
        del _kani_cache[model_name]
        # print(f"Cleared {model_name} from kani_cache.")

    if model_name in _hf_engine_cache:
        del _hf_engine_cache[model_name]
        # print(f"Cleared {model_name} from hf_engine_cache.")

    # Force Python's garbage collector to run
    gc.collect()

    # Tell PyTorch to release unused cached memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        # print("Cleared CUDA cache.")

In [ ]:
import json
import re # Already in your original code
def repair_json(content):
    
    # Clean up the content
    content = _remove_think_tags(content)
    content = _extract_json_from_codeblock(content)
    content = _fix_triple_quoted_strings(content)
    content = _extract_json_from_plain_text(content)
    content = _fix_unescaped_characters(content)
    content = _strip_formatting(content)

    return content

def _remove_think_tags(text):
    think_end = text.find('</think>')
    if think_end != -1:
        return text[think_end + 8:].strip()  # len('</think>') = 8
    return text

def _extract_json_from_codeblock(text):
    # Try to match ```json ... ``` first
    patterns = [
        r"(?s)```json\s*(.*?)\s*```",  # JSON code block
        r"(?s)```\s*(.*?)\s*```"        # Generic code block
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1).strip()
    
    return text

def _extract_json_from_plain_text(text: str) -> str:
    last_brace_pos = text.rfind('{')
    if last_brace_pos != -1:
        return text[last_brace_pos:]

    return text

def _fix_triple_quoted_strings(text):
    pattern = r'"(\w+)":\s*"""(.*?)"""'
    
    def escape_for_json(match):
        key = match.group(1)
        content = match.group(2)
        
        # Escape special characters for JSON
        replacements = [
            ('\\', '\\\\'),  # Backslashes first
            ('"', '\\"'),    # Quotes
            ('\n', '\\n'),   # Newlines
            ('\r', '\\r'),   # Carriage returns
            ('\t', '\\t'),   # Tabs
        ]
        
        for old, new in replacements:
            content = content.replace(old, new)
        
        return f'"{key}": "{content}"'
    
    return re.sub(pattern, escape_for_json, text, flags=re.DOTALL)

def _fix_unescaped_characters(text: str) -> str:
    # Set a limit to prevent infinite loops on unfixable errors
    max_attempts = 200
    
    for i in range(max_attempts):
        try:
            # Try to parse the text. If it works, we're done.
            json.loads(text)
            return text
        except json.JSONDecodeError as e:
            # On error, the parser tells us exactly where the problem is.
            # e.pos is the character index of the error.
            
            # Case 1: An unescaped newline or other control character in a string
            if "Invalid control character" in e.msg:
                # Replace the problematic character with its escaped version
                char_to_escape = text[e.pos]
                if char_to_escape == '\n':
                    escaped_char = '\\n'
                elif char_to_escape == '\r':
                    escaped_char = '\\r'
                elif char_to_escape == '\t':
                    escaped_char = '\\t'
                else:
                    # If it's some other control character, just remove it
                    escaped_char = ''
                
                text = text[:e.pos] + escaped_char + text[e.pos + 1:]

            # Case 2: An unescaped double quote in a string
            elif "Unterminated string" in e.msg:
                # This often means a '"' is inside a string without being escaped.
                # We'll try to find the quote just before the error and escape it.
                text = text[:e.pos - 1] + '\\' + text[e.pos - 1:]

            # If we can't fix it, break the loop and return the broken text
            else:
                break
    return text

def _strip_formatting(text):
    # Remove leading/trailing markdown json markers
    text = text.strip()
    if text.startswith("```json"):
        text = text[7:]  # len("```json") = 7
    if text.endswith("```"):
        text = text[:-3]
    
    # Remove surrounding quotes if present
    text = text.strip()
    if text.startswith("'") and text.endswith("'") and len(text) > 1:
        text = text[1:-1]
    
    return text.strip()


In [24]:
repaired_content = repair_json(text)
json.loads(repaired_content)

{'df': '(define (domain put-cds-in-safe)\n  (:requirements :strips :typing :negative-preconditions)\n  \n  (:types\n    object receptacle)\n  \n  (:predicates\n    (at ?o - object)\n    (in ?o ?r - receptacle)\n    (opened ?r - receptacle)\n    (closed ?r - receptacle))\n  \n  (:action GotoLocation\n    :parameters (?from - receptacle ?to - receptacle)\n    :precondition (and (at ?from) (not (at ?to)))\n    :effect (and (at ?to) (not (at ?from))))\n  \n  (:action OpenObject\n    :parameters (?r - receptacle)\n    :precondition (and (at ?r) (closed ?r))\n    :effect (and (opened ?r) (not (closed ?r))))\n  \n  (:action CloseObject\n    :parameters (?r - receptacle)\n    :precondition (and (at ?r) (opened ?r))\n    :effect (and (closed ?r) (not (opened ?r))))\n  \n  (:action PickupObject\n    :parameters (?o - object ?r - receptacle)\n    :precondition (and (at ?r) (in ?o ?r) (opened ?r))\n    :effect (and (at ?o) (not (in ?o ?r))))\n  \n  (:action PutObject\n    :parameters (?o - object 

In [18]:
from kani import Kani
from kani.engines.huggingface import HuggingEngine
import glob
import os
import json
from os.path import join as pjoin
from alfworld.info import ALFWORLD_DATA
from alfworld.agents.utils.misc import add_task_to_grammar
from alfworld.agents.environment.alfred_tw_env import AlfredExpert, AlfredDemangler, AlfredExpertType
import textworld
import textworld.gym


In [19]:
ALFWORLD_DATA

'/home/yl3427/.cache/alfworld'

In [20]:
### ========= Alfworld =========
# choose a problem to solve
problems = glob.glob(pjoin(ALFWORLD_DATA, "**", "initial_state.pddl"), recursive=True)
problems = [p for p in problems if "movable_recep" not in p]
if len(problems) == 0:
    raise ValueError(f"Can't find problem files in {ALFWORLD_DATA}. Did you run alfworld-data?")

In [21]:
len(problems)

6025

In [23]:
problems[3]

'/home/yl3427/.cache/alfworld/json_2.1.1/valid_unseen/pick_two_obj_and_place-CD-None-Safe-308/trial_T20190907_051013_060265/initial_state.pddl'

In [28]:
problem_id = 3
problems[problem_id]


'/home/yl3427/.cache/alfworld/json_2.1.1/valid_unseen/pick_two_obj_and_place-CD-None-Safe-308/trial_T20190907_051013_060265/initial_state.pddl'

In [29]:
problem = os.path.dirname(problems[3])

In [30]:
domain = pjoin(ALFWORLD_DATA, "logic", "alfred.pddl")
grammar = pjoin(ALFWORLD_DATA, "logic", "alfred.twl2")

GAME_LOGIC = {
        "pddl_domain": open(domain).read(),
        "grammar": open(grammar).read(),
    }

# load state and trajectory files
pddl_file = os.path.join(problem, 'initial_state.pddl') # pf
json_file = os.path.join(problem, 'traj_data.json')
with open(json_file, 'r') as f:
    traj_data = json.load(f)
GAME_LOGIC['grammar'] = add_task_to_grammar(GAME_LOGIC['grammar'], traj_data)

In [31]:
gamedata = dict(**GAME_LOGIC, pddl_problem=open(pddl_file).read())
gamefile = os.path.join(os.path.dirname(pddl_file), 'game.tw-pddl')

In [32]:
gamedata

{'pddl_domain': ";; Specification in PDDL of the Alfred domain\n;; Intended to be used with Fast Downward which supports PDDL 2.2 level 1 plus the :action-costs requirement from PDDL 3.1.\n\n(define (domain alfred)\n (:requirements\n    :adl\n    :action-costs\n    :typing\n )\n (:types\n  agent\n  location\n  receptacle\n  object\n  rtype\n  otype\n  )\n\n\n (:predicates\n    (atLocation ?a - agent ?l - location)                     ; true if the agent is at the location\n    (receptacleAtLocation ?r - receptacle ?l - location)      ; true if the receptacle is at the location (constant)\n    (objectAtLocation ?o - object ?l - location)              ; true if the object is at the location\n    (openable ?r - receptacle)                                ; true if a receptacle is openable\n    (opened ?r - receptacle)                                  ; true if a receptacle is opened\n    (inReceptacle ?o - object ?r - receptacle)                ; object ?o is in receptacle ?r\n    (isRecep

In [33]:
json.dump(gamedata, open(gamefile, "w")) #textworld register

In [34]:
# expert = AlfredExpert(expert_type=AlfredExpertType.PLANNER)
expert = AlfredExpert(expert_type=AlfredExpertType.HANDCODED)

In [35]:
request_infos = textworld.EnvInfos(
    won=True,
    admissible_commands=True,
    score=True,
    max_score=True,
    intermediate_reward=True,
    extras=["expert_plan"]
)
# reset environment starts here!
env_id = textworld.gym.register_game(
    gamefile,
    request_infos,
    max_episode_steps=1000000,
    wrappers=[AlfredDemangler(), expert]
)
env = textworld.gym.make(env_id)

obs, infos = env.reset()

In [36]:
print(obs)

-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 2, a desk 1, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.

Your task is to: put two cd in safe.


In [37]:
infos

{'intermediate_reward': None,
 'won': False,
 'admissible_commands': ['go to bed 1',
  'go to desk 1',
  'go to desk 2',
  'go to drawer 1',
  'go to drawer 2',
  'go to drawer 3',
  'go to drawer 4',
  'go to drawer 5',
  'go to drawer 6',
  'go to garbagecan 1',
  'go to laundryhamper 1',
  'go to safe 1',
  'go to shelf 1',
  'go to shelf 2',
  'go to shelf 3',
  'go to shelf 4',
  'go to shelf 5',
  'go to shelf 6',
  'help',
  'inventory',
  'look'],
 'score': None,
 'facts': [Proposition('objectatlocation', (Variable('keychain 2', 'object'), Variable('loc 23', 'location'))),
  Proposition('objectatlocation', (Variable('pen 3', 'object'), Variable('loc 23', 'location'))),
  Proposition('objectatlocation', (Variable('laptop 2', 'object'), Variable('loc 13', 'location'))),
  Proposition('objectatlocation', (Variable('cd 2', 'object'), Variable('loc 23', 'location'))),
  Proposition('objectatlocation', (Variable('pen 2', 'object'), Variable('loc 4', 'location'))),
  Proposition('obje

In [53]:
init_obs = obs.split('\n')[2]
print(init_obs)
goal = obs.split('\n')[-1]
valid_actions = infos["admissible_commands"]
valid_actions.remove('look')
valid_actions.remove('inventory')
valid_actions.remove('help')

You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 2, a desk 1, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.


In [54]:
print(goal)

Your task is to: put two cd in safe.


In [55]:
obs, reward, done, infos = env.step('go to shelf 5')

In [56]:
obs

'You arrive at shelf 5. On the shelf 5, you see a keychain 1.'

In [57]:
reward

0

In [59]:
infos

{'won': False,
 'max_score': None,
 'admissible_commands': ['examine shelf 5',
  'go to bed 1',
  'go to desk 1',
  'go to desk 2',
  'go to drawer 1',
  'go to drawer 2',
  'go to drawer 3',
  'go to drawer 4',
  'go to drawer 5',
  'go to drawer 6',
  'go to garbagecan 1',
  'go to laundryhamper 1',
  'go to safe 1',
  'go to shelf 1',
  'go to shelf 2',
  'go to shelf 3',
  'go to shelf 4',
  'go to shelf 6',
  'help',
  'inventory',
  'look',
  'take keychain 1 from shelf 5'],
 'facts': [Proposition('cancontain', (Variable('drawertype ', 'rtype'), Variable('pentype ', 'otype'))),
  Proposition('cancontain', (Variable('safetype ', 'rtype'), Variable('keychaintype ', 'otype'))),
  Proposition('cancontain', (Variable('shelftype ', 'rtype'), Variable('pentype ', 'otype'))),
  Proposition('cancontain', (Variable('desktype ', 'rtype'), Variable('pentype ', 'otype'))),
  Proposition('cancontain', (Variable('drawertype ', 'rtype'), Variable('creditcardtype ', 'otype'))),
  Proposition('can

In [8]:
import requests
def run_solver(domain_file, problem_file, solver = "dual-bfws-ffparser"):

    req_body = {"domain" : domain_file, "problem" : problem_file}

    # Send job request to solve endpoint
    solve_request_url=requests.post(f"https://solver.planning.domains:5001/package/{solver}/solve", json=req_body).json()
    print(solve_request_url)
    # Query the result in the job
    celery_result=requests.post('https://solver.planning.domains:5001' + solve_request_url['result'])
    print(celery_result.json())

    while celery_result.json().get("status","")== 'PENDING':
        # Query the result every 0.5 seconds while the job is executing
        celery_result=requests.post('https://solver.planning.domains:5001' + solve_request_url['result'])

    result = celery_result.json()['result']
    return result


In [1]:
# read domain.pddl
import os


with open("domain.pddl", "r") as f:
    domain_pddl = f.read()
with open("problem.pddl", "r") as f:
    problem_pddl = f.read()

In [9]:
run_solver(domain_pddl, problem_pddl)

{'result': '/check/10c0c439-b1ed-46c6-ba06-b478ea95df51?external=True'}
{'result': {'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': ''}, 'output_type': 'generic', 'stderr': "\ndomain: syntax error in line 4, ':PRECONDITIONS':\ndomain definition expected\n", 'stdout': ''}, 'status': 'ok'}


{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan',
 'output': {'plan': ''},
 'output_type': 'generic',
 'stderr': "\ndomain: syntax error in line 4, ':PRECONDITIONS':\ndomain definition expected\n",
 'stdout': ''}

In [2]:
import re

def format_pddl(pddl_text):
    """Format PDDL text with proper indentation and line breaks"""
    # Add line breaks after key PDDL keywords
    formatted = pddl_text.replace('(define', '\n(define')
    formatted = formatted.replace('(:requirements', '\n  (:requirements')
    formatted = formatted.replace('(:types', '\n  (:types')
    formatted = formatted.replace('(:action', '\n  (:action')
    formatted = formatted.replace(':parameters', '\n    :parameters')
    formatted = formatted.replace(':preconditions', '\n    :preconditions')
    formatted = formatted.replace(':effects', '\n    :effects')
    
    # Add line breaks before closing major sections
    formatted = formatted.replace('))', ')\n)')
    
    # Clean up extra whitespace and ensure proper spacing
    lines = formatted.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if line:
            cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)

# Format and print the domain_pddl
print("PDDL Domain (Formatted):")
print("=" * 50)
formatted_domain = format_pddl(domain_pddl)
print(formatted_domain)
print("=" * 50)

PDDL Domain (Formatted):
(define (domain exploration)
(:requirements :strips :typing)
(:types location direction)
(:action open-door
:parameters (?loc1 - location ?loc2 - location ?dir - direction)
:preconditions (and (at ?loc1) (door-closed ?loc1 ?loc2 ?dir)
)
:effects (door-open ?loc1 ?loc2 ?dir)
)
(:action move
:parameters (?from - location ?to - location ?dir - direction)
:preconditions (and (at ?from) (door-open ?from ?to ?dir)
)
:effects (at ?to)
))


In [8]:
import sys
# print(sys.path)
import time
from datetime import date
import csv
import json
import asyncio
import re

## module for server
# from kani import Kani
# from kani.engines.huggingface import HuggingEngine

import subprocess
import requests

import os
import json
import glob
import random
import argparse
from os.path import join as pjoin

from textworld_express import TextWorldExpressEnv
import textworld
import textworld.gym

from alfworld.info import ALFWORLD_DATA
from alfworld.agents.utils.misc import add_task_to_grammar
from alfworld.agents.environment.alfred_tw_env import AlfredExpert, AlfredDemangler, AlfredExpertType

from openai import OpenAI

In [89]:
env = TextWorldExpressEnv(envStepLimit=100)
NUM_LOCATIONS_lst = [3,5,7,9,11]
seed_lst = range(1,11)
NUM_LOCATIONS = NUM_LOCATIONS_lst[1]

In [94]:
env.load(gameName="coin", gameParams=f"numLocations={NUM_LOCATIONS},numDistractorItems=2, includeDoors=1, limitInventorySize=0")
obs, infos = env.reset(seed=seed_lst[0], gameFold="train", generateGoldPath=True)

In [141]:
env.getRunHistory()

{'properties': {'numDistractorItems': 2,
  'seed': 1,
  'numLocations': 5,
  'includeDoors': 1,
  'gameSet': 1,
  'limitInventorySize': 0},
 'finalScore': 0.0,
 'numSteps': 18,
 'history': [{'observation': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. \nTo the South you see a closed plain door. To the West you see a closed sliding patio door. ',
   'look': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it.

In [96]:
obs

'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. \nTo the South you see a closed plain door. To the West you see a closed sliding patio door. '

In [97]:
print(infos['observation'])

You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. 
To the South you see a closed plain door. To the West you see a closed sliding patio door. 


In [98]:
valid_actions = sorted(infos['validActions'])
# valid_actions.remove('look around') # 이거 왜뺐지
# valid_actions.remove('inventory') # 이거 왜뺐지
valid_actions

['close door to south',
 'close door to west',
 'inventory',
 'look around',
 'move south',
 'move west',
 'open door to south',
 'open door to west']

In [99]:
env.getGoldActionSequence()

['look around',
 'open door to south',
 'open door to west',
 'move west',
 'take coin']

In [138]:
action = 'move north'
obs, reward, done, infos = env.step(action)

In [ ]:
obs # 결국 문 말고는 다른걸 여는 선택지는 없네(drawer, cupboard, trashcan, fridge, ...etc.)

'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor. '

In [55]:
reward

0.0

In [45]:
done

True

In [140]:
infos

{'observation': 'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor. ',
 'look': 'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor

In [68]:
env.getRunHistory()['history']

[{'observation': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. There is also a coin. \nTo the South you see the corridor. To the East you see a closed plain door. ',
  'look': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room

In [ ]:
valid_actions = sorted(infos['validActions'])
valid_actions.remove('look around') # 이거 왜뺐지
valid_actions.remove('inventory') # 이거 왜뺐지

print(f"Observations: {obs} \n") 
print(f"Gold path: {env.getGoldActionSequence()} \n")
print(f"Valid Actions: {valid_actions} \n")
print(f"taskDescription: {infos['taskDescription']} \n")

# actions = ['open door to south', 'move south', 'open door to west', 'move west', 'move east', 'move north', 'open door to west', 'move west', 'move east', 'move south', 'move south', 'move north', 'move east', 'open door to north', 'move north', 'take coin']
actions = ['open door to south']

for action in actions:
    print('>', action)
    obs, reward, done, infos = env.step(action)
    print(obs)

print(infos['done'])

In [6]:
domain_file = open(f'domain.pddl').read()
problem_file = open(f'problem.pddl').read()
domain_file

'(define (domain exploration)\n  (:requirements :strips :typing)\n  \n  (:types\n    location direction\n  )\n  \n  (:predicates\n    (at ?loc - location)\n    (door-open ?loc1 - location ?loc2 - location ?dir - direction)\n    (door-closed ?loc1 - location ?loc2 - location ?dir - direction)\n  )\n  \n  (:action open-door\n    :parameters (?loc1 - location ?loc2 - location ?dir - direction)\n    :precondition (and (at ?loc1) (door-closed ?loc1 ?loc2 ?dir))\n    :effect (and (door-open ?loc1 ?loc2 ?dir) (not (door-closed ?loc1 ?loc2 ?dir)))\n  )\n  \n  (:action move\n    :parameters (?from - location ?to - location ?dir - direction)\n    :precondition (and (at ?from) (door-open ?from ?to ?dir))\n    :effect (and (at ?to) (not (at ?from)))\n  )\n)'

In [9]:
max_retries=3

req_body = {"domain" : domain_file, "problem" : problem_file}

retries = 0

while retries < max_retries:
    try:
        # Send job request to solve endpoint
        solve_request_url = requests.post(
            f"https://solver.planning.domains:5001/package/dual-bfws-ffparser/solve", 
            json=req_body
        ).json()

        print(solve_request_url)

        # Query the result in the job
        celery_result = requests.post(
            'https://solver.planning.domains:5001' + solve_request_url['result']
        )
        print(celery_result.json().get("status", "?"))
        while celery_result.json().get("status", "") == 'PENDING':
            time.sleep(0.5)
            celery_result = requests.post(
                'https://solver.planning.domains:5001' + solve_request_url['result']
            )
        
        print(celery_result.json()['result'])
        break

    except Exception as e:
        print(f"Error encountered: {e}. Retrying in 5 seconds... (Attempt {retries+1}/{max_retries})")
        retries += 1
        time.sleep(5)

{'result': '/check/158419d0-5072-47a0-8e7f-d24736a9fd7a?external=True'}
ok
{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': '(OPEN-DOOR KITCHEN WEST_ROOM WEST)\n(MOVE KITCHEN WEST_ROOM WEST)\n'}, 'output_type': 'generic', 'stderr': '', 'stdout': ' --- OK.\n Match tree built with 4 nodes.\n\nPDDL problem description loaded: \n\tDomain: EXPLORATION\n\tProblem: EXPLORATION-PROBLEM\n\t#Actions: 4\n\t#Fluents: 7\nGoals found: 1\nGoals_Edges found: 1\nStarting search with 1-BFWS...\n--[1 / 0]--\n--[1 / 1]--\n--[0 / 0]--\n--[0 / 2]--\nTotal time: 6.19997e-05\nNodes generated during search: 5\nNodes expanded during search: 2\nPlan found with cost: 2\nFast-BFS search completed in 6.19997e-05 secs\n'}


In [11]:
celery_result.json()['result']['output']

{'plan': '(OPEN-DOOR KITCHEN WEST_ROOM WEST)\n(MOVE KITCHEN WEST_ROOM WEST)\n'}

In [12]:
def map_actions(action):
    actions = action.lower().replace("(", "").replace(")", "").split('\n')
    action_lst = []
    for act in actions:
        if "open" in act and "door" in act:
            direction = act.split(' ')[-1]
            action_lst.append(f'open door to {direction}')
        elif "move" in act:
            action_lst.append(f"move {act.split(' ')[-1]}")
    if len(action_lst) == 0:
        return None
    return action_lst

In [13]:
map_actions(celery_result.json()['result']['output']['plan'])

['open door to west', 'move west']

In [15]:
"'hello".startswith("'")

True